In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn 
from torch.utils.data import random_split, DataLoader, TensorDataset 
import flwr as fl
from flwr.common import Metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import OrderedDict
from typing import List, Tuple
import sys
import time
import warnings
warnings.simplefilter('ignore')
import platform, cpuinfo, GPUtil, psutil

In [2]:
# print(f"OS: {platform.uname().system} {platform.uname().release}")
# print(f"CPU: {cpuinfo.get_cpu_info()['brand_raw']}")
# print(f"GPU: {GPUtil.getGPUs()[0].name}")
# print(f"Memory: {psutil.virtual_memory().total / (1024 ** 3):.2f} GB")

In [3]:
print("Python version:", sys.version)
print("Version info:", sys.version_info)

Python version: 3.11.4 (tags/v3.11.4:d2340ef, Jun  7 2023, 05:45:37) [MSC v.1934 64 bit (AMD64)]
Version info: sys.version_info(major=3, minor=11, micro=4, releaselevel='final', serial=0)


In [4]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}")

Training on cpu using PyTorch 2.0.1+cpu and Flower 1.5.0


In [5]:
NUM_CLIENTS = 3
EPOCHS = 10
ROUNDS = 5

BATCH_SIZE = 32
IN_FEATURES = 3
HIDDEN_LAYERS = 80
OUT_FEATURES = 2

In [6]:
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
def load_datasets():
    df = pd.read_csv('./datasets/label_data.csv')
    df = df.rename(columns={'label': 'target'})
    
    sm = RandomOverSampler(random_state=42)
    X3, y3 = sm.fit_resample(df.iloc[:, :-1], df.iloc[:, -1])
    df = pd.DataFrame(X3)
    df['target'] = y3
    
    feature = df.iloc[:, :-1]
    target = df.loc[:, 'target']
    feature = torch.Tensor(feature.to_numpy())
    target = torch.tensor(target.to_numpy())
    tensor_data = TensorDataset(feature, target)
    
    split_ratio = 0.8
    train_split = int(len(feature) * split_ratio)
    test_split = len(feature) - train_split
    while train_split % NUM_CLIENTS != 0:
        train_split -= 1
        test_split += 1
    train_set, test_set = random_split(tensor_data, [train_split, test_split])  

    # split_index = int(len(df) * split_ratio)
    # train_set = tensor_data.iloc[:split_index, :]
    # test_set = tensor_data.iloc[split_index:, :]
    
    part_size = len(train_set) // NUM_CLIENTS
    length = [part_size] * NUM_CLIENTS  # lengths for each client
    
    # Split the test set evenly into thirds, removing the remainders
    # random_choose = np.random.choice(train_set.index, (len(train_set) % NUM_CLIENTS), replace=False)
    # train_set = train_set.drop(random_choose)
    
    datasets = random_split(train_set, length, generator=torch.Generator().manual_seed(42))
    
    train_loader = []
    val_loader = []
    
    for data in datasets:
        val_length = len(data) // 10  # 90% : 10%
        
        train_length = len(data) - val_length
        length = [train_length, val_length]
        train_data, val_data = random_split(data, length, generator=torch.Generator().manual_seed(42))
        
        train_loader.append(DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True))
        val_loader.append(DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True))
    
    test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)
    return train_loader, val_loader, test_loader
    
train_loader, val_loader, test_loader = load_datasets()

In [7]:
class Network(nn.Module):
    def __init__(self, IN_FEATURES, HIDDEN_LAYERS, OUT_FEATURES):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(IN_FEATURES, HIDDEN_LAYERS), 
            nn.ReLU(),
            nn.Linear(HIDDEN_LAYERS, HIDDEN_LAYERS), 
            nn.ReLU(),
            nn.Linear(HIDDEN_LAYERS, OUT_FEATURES), 
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
def train(model, train_loader, epochs):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    model.train()
    
    for epoch in range(1, epochs + 1): 
        correct, total, epoch_loss = 0, 0, 0.0
        
        for feature, target in train_loader:
            feature, target = feature.to(DEVICE), target.to(DEVICE)
            optimizer.zero_grad()
            output = model(feature)
            train_loss = criterion(output, target)
            train_loss.backward()
            optimizer.step()
            
            epoch_loss += train_loss
            total += target.size(0)
            correct += (torch.max(output.data, 1)[1] == target).sum().item()
            
        epoch_loss /= len(train_loader.dataset)
        epoch_accuracy = correct / total
        
        print(f"Epoch {epoch}/{EPOCHS}: train loss: {epoch_loss:.4f}, accuracy: {epoch_accuracy:.4f}")

In [9]:
def test(model, test_loader):
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    actual_labels = []
    predicted_labels = []
    
    model.eval()
 
    with torch.no_grad():
        for feature, target in test_loader:
            feature, target = feature.to(DEVICE), target.to(DEVICE)
            output = model(feature)
            _, predicted = torch.max(output.data, 1)
            
            loss += criterion(output, target).item()
            # total += target.size(0)
            # correct += (predicted == target).sum().item()
            
            actual_labels.extend(target.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())
            
        loss /= len(test_loader.dataset)
        # accuracy = correct / total
        # return loss, accuracy
        
    accuracy = accuracy_score(actual_labels, predicted_labels)
    precision = precision_score(actual_labels, predicted_labels, average='weighted')
    recall = recall_score(actual_labels, predicted_labels, average='weighted')
    f1 = f1_score(actual_labels, predicted_labels, average='weighted')
    
    print(f'Loss: {accuracy:.4f}   Accuracy: {accuracy:.4f}   Precision: {precision:.4f}   Recall: {recall:.4f}   F1-Score: {f1:.4f}')

In [10]:
# PyTorch model testing
# train_loader = train_loader[1]
# val_loader = val_loader[1]
# model = Network(IN_FEATURES, HIDDEN_LAYERS, OUT_FEATURES).to(DEVICE)
# 
# train(model, train_loader, EPOCHS)
# print('Final test performance:')
# test(model, val_loader)

# loss, accuracy = test(model, val_loader)
# print(f"Final test performance: \n\tloss: {loss:.8f}, accuracy: {accuracy:.8f}")

In [11]:
def get_parameters(model) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in model.state_dict().items()]

def set_parameters(model, parameters: List[np.ndarray]):
    params_dict = zip(model.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    model.load_state_dict(state_dict, strict=True)

In [12]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, val_loader):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader

    def get_parameters(self, config):
        return get_parameters(self.model)

    def fit(self, parameters, config):
        set_parameters(self.model, parameters)
        train(self.model, self.train_loader, epochs=EPOCHS)
        return get_parameters(self.model), len(self.train_loader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.model, parameters)
        loss, accuracy = test(self.model, self.val_loader)
        return float(loss), len(self.val_loader), {'accuracy: ': float(accuracy)}

In [13]:
def client_fn(cid: str) -> FlowerClient:
    print('Client:', cid)
    model = Network(IN_FEATURES, HIDDEN_LAYERS, OUT_FEATURES).to(DEVICE)
    train_loader, val_loader, test_loader = load_datasets()
    train_loader = train_loader[int(cid)]
    val_loader = val_loader[int(cid)]
    return FlowerClient(model, train_loader, val_loader)

In [14]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    accuracies = [num_examples * m['accuracy'] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]
    return {'accuracy': sum(accuracies) / sum(examples)}

In [15]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=1.0,
    min_fit_clients=NUM_CLIENTS,
    min_evaluate_clients=NUM_CLIENTS,
    min_available_clients=NUM_CLIENTS,
    evaluate_metrics_aggregation_fn=weighted_average,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=ROUNDS),
    strategy=strategy,
)

INFO flwr 2023-10-24 18:28:03,898 | app.py:175 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-10-24 18:28:06,660	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-10-24 18:28:09,407 | app.py:210 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 12915624347.0, 'object_store_memory': 6457812172.0, 'accelerator_type:G': 1.0, 'GPU': 1.0, 'node:__internal_head__': 1.0, 'CPU': 20.0}
INFO flwr 2023-10-24 18:28:09,407 | app.py:218 | No `client_resources` specified. Using minimal resources for clients.
INFO flwr 2023-10-24 18:28:09,407 | app.py:224 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2023-10-24 18:28:09,415 | app.py:270 | Flower VCE: Creating VirtualClientEngineActorPool with 20 actors
INFO flwr 2023-10-24 18:28:09,415 | server.py:89 | Initializing global parameters
INFO flwr 2023-10-24 18:28:09,419 | server.py:276 | Requesting initial parameters fro

(DefaultActor pid=16768) Client: 1


INFO flwr 2023-10-24 18:28:19,767 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-10-24 18:28:19,767 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-10-24 18:28:19,767 | server.py:104 | FL starting
DEBUG flwr 2023-10-24 18:28:19,767 | server.py:222 | fit_round 1: strategy sampled 3 clients (out of 3)


(DefaultActor pid=16768) Client: 1
(DefaultActor pid=16768) Epoch 1/10: train loss: 0.0209, accuracy: 0.5850
(DefaultActor pid=3080) Client: 0 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(DefaultActor pid=16768) Epoch 2/10: train loss: 0.0202, accuracy: 0.6274 [repeated 3x across cluster]
(DefaultActor pid=16768) Epoch 3/10: train loss: 0.0201, accuracy: 0.6318 [repeated 3x across cluster]
(DefaultActor pid=16768) Epoch 4/10: train loss: 0.0201, accuracy: 0.6333 [repeated 3x across cluster]
(DefaultActor pid=16768) Epoch 5/10: train loss: 0.0200, accuracy: 0.6353 [repeated 3x across cluster]
(DefaultActor pid=16768) Epoch 6/10: train loss: 0.0200, accuracy: 0.6364 [repeated 3x across cluster]
(DefaultActor pid=16768) Epoch 7/10: train loss: 0.0200, accuracy: 0.6376 [repeated 3x across cluster]
(DefaultActor p

DEBUG flwr 2023-10-24 18:29:26,766 | server.py:236 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-10-24 18:29:26,770 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-10-24 18:29:26,770 | server.py:173 | evaluate_round 1: strategy sampled 3 clients (out of 3)


...

ERROR flwr 2023-10-24 18:32:25,223 | ray_client_proxy.py:147 | Traceback (most recent call last):
  File "D:\Python\Python311\Lib\site-packages\flwr\simulation\ray_transport\ray_client_proxy.py", line 140, in _submit_job
    res = self.actor_pool.get_client_result(self.cid, timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Python\Python311\Lib\site-packages\flwr\simulation\ray_transport\ray_actor.py", line 402, in get_client_result
    return self._fetch_future_result(cid)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Python\Python311\Lib\site-packages\flwr\simulation\ray_transport\ray_actor.py", line 288, in _fetch_future_result
    res_cid, res = ray.get(future)  # type: (str, ClientRes)
                   ^^^^^^^^^^^^^^^
  File "D:\Python\Python311\Lib\site-packages\ray\_private\auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "D:\Python\Python311\Lib\site-packages\ray\_priv

(DefaultActor pid=5944) Loss: 0.6529   Accuracy: 0.6529   Precision: 0.6529   Recall: 0.6529   F1-Score: 0.6528
(DefaultActor pid=3080) Epoch 1/10: train loss: 0.0197, accuracy: 0.6479 [repeated 3x across cluster]
(DefaultActor pid=16768) Client: 1 [repeated 5x across cluster]
(DefaultActor pid=16768) Loss: 0.6581   Accuracy: 0.6581   Precision: 0.6582   Recall: 0.6581   F1-Score: 0.6580 [repeated 2x across cluster]
(DefaultActor pid=3080) Epoch 2/10: train loss: 0.0197, accuracy: 0.6484 [repeated 3x across cluster]
(DefaultActor pid=3080) Epoch 3/10: train loss: 0.0197, accuracy: 0.6478 [repeated 3x across cluster]
(DefaultActor pid=16768) Epoch 4/10: train loss: 0.0197, accuracy: 0.6501 [repeated 3x across cluster]
(DefaultActor pid=3080) Epoch 5/10: train loss: 0.0197, accuracy: 0.6489 [repeated 3x across cluster]
(DefaultActor pid=3080) Epoch 6/10: train loss: 0.0197, accuracy: 0.6488 [repeated 3x across cluster]
(DefaultActor pid=16768) Epoch 7/10: train loss: 0.0197, accuracy: 0.

DEBUG flwr 2023-10-24 18:33:23,514 | server.py:236 | fit_round 5 received 3 results and 0 failures
DEBUG flwr 2023-10-24 18:33:23,519 | server.py:173 | evaluate_round 5: strategy sampled 3 clients (out of 3)


(DefaultActor pid=5944) Client: 1


ERROR flwr 2023-10-24 18:33:24,282 | ray_client_proxy.py:147 | Traceback (most recent call last):
  File "D:\Python\Python311\Lib\site-packages\flwr\simulation\ray_transport\ray_client_proxy.py", line 140, in _submit_job
    res = self.actor_pool.get_client_result(self.cid, timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Python\Python311\Lib\site-packages\flwr\simulation\ray_transport\ray_actor.py", line 402, in get_client_result
    return self._fetch_future_result(cid)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Python\Python311\Lib\site-packages\flwr\simulation\ray_transport\ray_actor.py", line 288, in _fetch_future_result
    res_cid, res = ray.get(future)  # type: (str, ClientRes)
                   ^^^^^^^^^^^^^^^
  File "D:\Python\Python311\Lib\site-packages\ray\_private\auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "D:\Python\Python311\Lib\site-packages\ray\_priv